In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor 
import sklearn.metrics as sm

In [ ]:
df = pd.read_csv("C:/Users/hp/Downloads/lab_11/events_train.tsv",delimiter = "\t")

In [ ]:
print(df.isnull().values.any())

In [ ]:
len(df)

In [ ]:
df1 = pd.read_csv("C:/Users/hp/Downloads/lab_11/prediction_trials.tsv",delimiter = "\t")

In [ ]:
df1['diff_lat'] = df1['nw_lat']-df1['se_lat']

In [ ]:
df1['diff_lon'] = df1['se_lon']-df1['nw_lon']

In [ ]:
df1.head()

In [ ]:
df1['diff_lon'].mean()

In [ ]:
df1['diff_lat'].mean()

In [ ]:
rightmost = df1['se_lon'].max()
rightmost

In [ ]:
leftmost = df1['nw_lon'].min()
leftmost

In [ ]:
topmost=df1['nw_lat'].max()
topmost

In [ ]:
bottommost = df1['se_lat'].min()
bottommost 

In [ ]:
(topmost-bottommost)/0.010



In [ ]:
(rightmost-leftmost)/0.010

In [ ]:
interval = 0.01
lat=bottommost
latitudes = list()
latitudes.append(lat)

for i in range(0,61):
    lat = lat+interval
    latitudes.append(lat)
    

In [ ]:
interval = 0.01
lon=leftmost
longitudes = list()
longitudes.append(lon)

for i in range(0,84):
    lon = lon+interval
    longitudes.append(lon)

In [ ]:
print(len(longitudes))
print(len(latitudes))

In [ ]:
geo_box=pd.DataFrame(index=[0], columns=['se_lat','nw_lat','se_lon','nw_lon'])

In [ ]:
index=0
for i in range(0,61):
    for j in range(0,84):
            
            geo_box.ix[index,'nw_lat'] = latitudes[i+1]
            geo_box.ix[index,'se_lat'] = latitudes[i]
            geo_box.ix[index,'nw_lon'] = longitudes[j]
            geo_box.ix[index,'se_lon'] = longitudes[j+1]
            index=index+1
    

In [ ]:
geo_box.to_csv("C:/Users/hp/Downloads/lab_11/geo_box.csv", encoding = "utf-8",index=False)

In [ ]:
len(geo_box)

In [ ]:
geo_box.head()

In [ ]:
df['timestamp_date'], df['timestamp_time'] = df['created_tstamp'].str.split('T', 1).str


In [ ]:
df.head()

In [ ]:
df['year'],df['month'],df['date']= df['timestamp_date'].str.split('-').str


In [ ]:
years = df['year'].unique()
years

In [ ]:
df=df.fillna(-1)


In [ ]:
df['year'] = df['year'].apply(lambda x : int(x))

In [ ]:
df2 = df[~(df['year']<0)]
len(df2)

In [ ]:
df2 = df2[~(df2['year']>2014)]
len(df2)

In [ ]:
df.to_csv("C:/Users/hp/Downloads/lab_11/events_date_time.csv")

In [ ]:
df2.to_csv("C:/Users/hp/Downloads/lab_11/events_date_time_filtered.csv")

In [ ]:
df2['event_type'].unique()

In [ ]:
events_of_interest = ['accidentsAndIncidents','roadwork','precipitation','deviceStatus','trafficConditions','obstruction']

In [ ]:
df3 = df2[df['event_type'].isin(events_of_interest) ]

In [ ]:
df.head(2)

In [ ]:
df3.to_csv("/home/swati/Downloads/events_only_6.csv", encoding = "utf-8")

In [ ]:
df3 = pd.read_csv("C:/Users/hp/Downloads/lab_11/events_only_6.csv", encoding = "ISO-8859-1")

In [ ]:
geo_box.head()

In [ ]:
geo_box['nw_lat'] = geo_box['nw_lat'].convert_objects(convert_numeric =True)
geo_box['se_lat'] = geo_box['se_lat'].convert_objects(convert_numeric =True)
geo_box['nw_lon'] = geo_box['nw_lon'].convert_objects(convert_numeric =True)
geo_box['se_lon'] = geo_box['se_lon'].convert_objects(convert_numeric =True)

df3['latitude'] = df3['latitude'].convert_objects(convert_numeric =True)
df3['longitude'] = df3['longitude'].convert_objects(convert_numeric =True)


In [ ]:
geo_combined_data = pd.DataFrame()

In [ ]:
len(geo_box)

In [ ]:
for index in range(0,50001):
    
    all_within_se = df3[df3['latitude']>=geo_box.ix[index,'se_lat']]
    all_within_lat = all_within_se[all_within_se['latitude']<= geo_box.ix[index,'nw_lat']]
    all_within_se_lon = all_within_lat[all_within_lat['longitude'] <= geo_box.ix[index,'se_lon']]
    all_points_in_box = all_within_se_lon[all_within_se_lon['longitude']>=geo_box.ix[index,'nw_lon']]
    
    geo_combined_data_row = pd.DataFrame(index=[0], columns=['se_lat','nw_lat','se_lon','nw_lon',\
                                         'accidentsAndIncidents','roadwork','precipitation','deviceStatus',\
                                                                   'trafficConditions','obstruction'])
    #geo_combined_data_row = pd.concat([all_points_in_box['event_type'],all_points_in_box['year'], \
                                  #all_points_in_box['month'] , all_points_in_box['created_tstamp']],axis=1  )
    
   
    geo_combined_data_row.ix[0,'se_lat'] = geo_box.ix[index,'se_lat']
    geo_combined_data_row.ix[0,'nw_lat'] = geo_box.ix[index,'nw_lat'] 
    geo_combined_data_row.ix[0,'se_lon'] = geo_box.ix[index,'se_lon']
    geo_combined_data_row.ix[0,'nw_lon'] = geo_box.ix[index,'nw_lon']
    
   
    grouped = all_points_in_box.groupby(['event_type', 'month']).size().reset_index(name='count')
   
    for i in range(1,13):
        geo_combined_data_row['month'] = i
        all_points_for_month = grouped[grouped['month'] == i]
        geo_combined_data_row.ix[0,'accidentsAndIncidents']=len(all_points_for_month[all_points_for_month['event_type'] == 'accidentsAndIncidents'])
        geo_combined_data_row.ix[0,'roadwork']=len(all_points_for_month[all_points_for_month['event_type'] == 'roadwork'])
        geo_combined_data_row.ix[0,'precipitation']=len(all_points_for_month[all_points_for_month['event_type'] == 'precipitation'])
        geo_combined_data_row.ix[0,'deviceStatus']=len(all_points_for_month[all_points_for_month['event_type'] == 'deviceStatus'])
        geo_combined_data_row.ix[0,'trafficConditions']=len(all_points_for_month[all_points_for_month['event_type'] == 'trafficConditions'])
        geo_combined_data_row.ix[0,'obstruction']=len(all_points_for_month[all_points_for_month['event_type'] == 'obstruction'])
    
    
    geo_combined_data = pd.concat([geo_combined_data,geo_combined_data_row])
    
    

In [ ]:
geo_combined_data.to_csv("C:/Users/hp/Downloads/lab_11/geo_combined_data_final.csv")

In [ ]:
geo_combined_data = pd.DataFrame()

In [ ]:
df3['year'] = df3['year'].apply(lambda x : int(x))


In [ ]:
df4 =  df3[df3['year']<2014]

In [ ]:
df4.to_csv("C:/Users/hp/Downloads/lab_11/only_till_2013.csv", encoding = "utf-8")

In [ ]:
df4 = pd.read_csv("C:/Users/hp/Downloads/lab_11/only_till_2013.csv", encoding = "ISO-8859-1")

In [ ]:
geo_combined_data = pd.DataFrame()
for index in range(0,len(geo_box)):
    #if(index%500 == 0):
    #    filename = 'C:/Users/hp/Downloads/lab_11/parts/geo_combined_data_final'+str(index)+'.csv'
    #    geo_combined_data.to_csv(filename, index = False)
    #    geo_combined_data = pd.DataFrame()
       
    if(index%500 ==0):
        print(index)
    all_within_se = df4[df4['latitude']>=geo_box.ix[index,'se_lat']]
    all_within_lat = all_within_se[all_within_se['latitude']<= geo_box.ix[index,'nw_lat']]
    all_within_se_lon = all_within_lat[all_within_lat['longitude'] <= geo_box.ix[index,'se_lon']]
    all_points_in_box = all_within_se_lon[all_within_se_lon['longitude']>=geo_box.ix[index,'nw_lon']]
    
    geo_combined_data_row = pd.DataFrame(index=[0], columns=['se_lat','nw_lat','se_lon','nw_lon',\
                                         'accidentsAndIncidents','roadwork','precipitation','deviceStatus',\
                                                                   'trafficConditions','obstruction'])
    #geo_combined_data_row = pd.concat([all_points_in_box['event_type'],all_points_in_box['year'], \
                                  #all_points_in_box['month'] , all_points_in_box['created_tstamp']],axis=1  )
    
   
    geo_combined_data_row.ix[0,'se_lat'] = geo_box.ix[index,'se_lat']
    geo_combined_data_row.ix[0,'nw_lat'] = geo_box.ix[index,'nw_lat'] 
    geo_combined_data_row.ix[0,'se_lon'] = geo_box.ix[index,'se_lon']
    geo_combined_data_row.ix[0,'nw_lon'] = geo_box.ix[index,'nw_lon']
    
   
    grouped = all_points_in_box.groupby(['event_type', 'month']).size().reset_index(name='count')
   
    
    for i in range(1,13):
        
        geo_combined_data_row.ix[0,'month'] = i
        all_points_for_month = grouped[grouped['month'] == i]
        
        all = all_points_for_month[all_points_for_month['event_type'] == 'accidentsAndIncidents']
        if(len(all>0)):
            geo_combined_data_row.ix[0,'accidentsAndIncidents']=all['count'].values[0]
        else:
            geo_combined_data_row.ix[0,'accidentsAndIncidents']=0
        
        all = all_points_for_month[all_points_for_month['event_type'] == 'roadwork']
        if(len(all>0)):
            geo_combined_data_row.ix[0,'roadwork']=all['count'].values[0]
        else:
            geo_combined_data_row.ix[0,'roadwork']=0
            
        all =  all_points_for_month[all_points_for_month['event_type'] == 'precipitation']
        if(len(all>0)):
            geo_combined_data_row.ix[0,'precipitation']=all['count'].values[0]
        else:
            geo_combined_data_row.ix[0,'precipitation']=0
        
        all =  all_points_for_month[all_points_for_month['event_type'] == 'deviceStatus']
        if(len(all>0)):
            geo_combined_data_row.ix[0,'deviceStatus']=all['count'].values[0]
        else:
            geo_combined_data_row.ix[0,'deviceStatus']=0
        
        all =  all_points_for_month[all_points_for_month['event_type'] == 'trafficConditions']
        if(len(all>0)):
            geo_combined_data_row.ix[0,'trafficConditions']=all['count'].values[0]
        else:
            geo_combined_data_row.ix[0,'trafficConditions']=0
        
        all =  all_points_for_month[all_points_for_month['event_type'] == 'obstruction']
        if(len(all>0)):
            geo_combined_data_row.ix[0,'obstruction']=all['count'].values[0]
        else:
            geo_combined_data_row.ix[0,'obstruction']=0
        
        geo_combined_data = pd.concat([geo_combined_data,geo_combined_data_row])
        
        
geo_combined_data.to_csv("C:/Users/hp/Downloads/lab_11/parts/last_part.csv", index = False)

In [ ]:
geo_combined_data.to_csv("C:/Users/hp/Downloads/lab_11/geo_combined_data_month_test_set_final.csv")

In [ ]:
geo_combined_data = pd.DataFrame()

In [ ]:
df5 =  df3[df3['year'] == 2014]

In [ ]:
for index in range(0,len(geo_box)):
    
    all_within_se = df5[df5['latitude']>=geo_box.ix[index,'se_lat']]
    all_within_lat = all_within_se[all_within_se['latitude']<= geo_box.ix[index,'nw_lat']]
    all_within_se_lon = all_within_lat[all_within_lat['longitude'] <= geo_box.ix[index,'se_lon']]
    all_points_in_box = all_within_se_lon[all_within_se_lon['longitude']>=geo_box.ix[index,'nw_lon']]
    
    geo_combined_data_row = pd.DataFrame(index=[0], columns=['se_lat','nw_lat','se_lon','nw_lon',\
                                         'accidentsAndIncidents','roadwork','precipitation','deviceStatus',\
                                                                   'trafficConditions','obstruction'])
    #geo_combined_data_row = pd.concat([all_points_in_box['event_type'],all_points_in_box['year'], \
                                  #all_points_in_box['month'] , all_points_in_box['created_tstamp']],axis=1  )
    
   
    geo_combined_data_row.ix[0,'se_lat'] = geo_box.ix[index,'se_lat']
    geo_combined_data_row.ix[0,'nw_lat'] = geo_box.ix[index,'nw_lat'] 
    geo_combined_data_row.ix[0,'se_lon'] = geo_box.ix[index,'se_lon']
    geo_combined_data_row.ix[0,'nw_lon'] = geo_box.ix[index,'nw_lon']
    
   
    grouped = all_points_in_box.groupby(['event_type', 'month']).size().reset_index(name='count')
   
    for i in range(1,13):
        geo_combined_data_row['month'] = i
        all_points_for_month = grouped[grouped['month'] == i]
        geo_combined_data_row.ix[0,'accidentsAndIncidents']=len(all_points_for_month[all_points_for_month['event_type'] == 'accidentsAndIncidents'])
        geo_combined_data_row.ix[0,'roadwork']=len(all_points_for_month[all_points_for_month['event_type'] == 'roadwork'])
        geo_combined_data_row.ix[0,'precipitation']=len(all_points_for_month[all_points_for_month['event_type'] == 'precipitation'])
        geo_combined_data_row.ix[0,'deviceStatus']=len(all_points_for_month[all_points_for_month['event_type'] == 'deviceStatus'])
        geo_combined_data_row.ix[0,'trafficConditions']=len(all_points_for_month[all_points_for_month['event_type'] == 'trafficConditions'])
        geo_combined_data_row.ix[0,'obstruction']=len(all_points_for_month[all_points_for_month['event_type'] == 'obstruction'])
    
    
    geo_combined_data = pd.concat([geo_combined_data,geo_combined_data_row])
    
    

In [ ]:
geo_combined_data.to_csv("C:/Users/hp/Downloads/lab_11/geo_combined_data_month_validation_set.csv")

In [2]:
df5 = pd.read_csv("C:/Users/hp/Downloads/geo_combined_data_month_test_set_final.csv", encoding = "ISO-8859-1")

In [3]:
df6 = pd.read_csv("C:/Users/hp/Downloads/geo_combined_data_validation.csv", encoding = "ISO-8859-1")

In [4]:
validation_sample = pd.DataFrame()

In [5]:
train_sample = pd.DataFrame()
train_sample_labels = pd.DataFrame()

In [6]:
train_sample['se_lat'] = df5.ix[:,'se_lat']
train_sample['nw_lat'] = df5.ix[:,'nw_lat']
train_sample['se_lon'] = df5.ix[:,'se_lon']
train_sample['nw_lon'] = df5.ix[:,'nw_lon']
train_sample['month'] = df5.ix[:,'month']

In [7]:
train_sample_labels['accidentsAndIncidents'] = df5.ix[:,'accidentsAndIncidents'] 



In [8]:
validation_sample['se_lat'] = df6.ix[:,'se_lat']
validation_sample['nw_lat'] = df6.ix[:,'nw_lat']
validation_sample['se_lon'] = df6.ix[:,'se_lon']
validation_sample['nw_lon'] = df6.ix[:,'nw_lon']
validation_sample['month'] = df6.ix[:,'month']

In [ ]:
train_sample_labels.head()

In [ ]:
model = SGDRegressor(eta0 = 0.000004,penalty='l1').fit(train_sample,train_sample_labels)

In [ ]:
predictions = model.predict(validation_sample)

In [ ]:
y_true

In [ ]:
predictions

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_pred =predictions1

In [ ]:
y_true = df6.ix[:,'accidentsAndIncidents']
y_true= y_true.values


In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
type(predictions)

In [ ]:
type(y_true)

In [ ]:
predictions1 = predictions.astype(int)

In [ ]:
y_true

In [ ]:
predictions1

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(train_sample, train_sample_labels)



In [ ]:
predicted = model.predict(validation_sample)


In [ ]:
predicted

In [ ]:
import math
def validation_score():
    y_pred = predictions
    y_result = (y_pred - y_true)**2
    y = y_result.mean()
    return math.sqrt(y)

In [ ]:
validation_score()

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear').fit(train_sample, train_sample_labels)

In [ ]:
predicted = model.predict(validation_sample)


In [9]:
from sklearn.ensemble import RandomForestClassifier


In [10]:
num_trees = 10
max_features = 5

In [11]:
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features).fit(train_sample, train_sample_labels)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


In [12]:
predictions_random = model.predict(validation_sample)

MemoryError: 